In [1]:
import numpy as np
import pandas as pd
from numpy.random import rand
import gzip
import json
import ast
from scipy.cluster.vq import kmeans2
import matplotlib.pyplot as plt

In [2]:
mu = 0
sigma = 1.0
A = np.random.normal(mu, sigma, (10,300))
A

array([[ 1.49531783,  1.30287202, -0.34395735, ...,  0.13080379,
         0.83533574, -0.38015671],
       [-1.38729941, -1.45238562,  0.57005743, ..., -0.72508667,
         0.51879238,  0.51545811],
       [ 0.28205511, -0.43806339, -1.46566083, ..., -0.32824124,
        -2.63896932,  0.51711023],
       ...,
       [ 1.01155266,  1.47383056, -2.30059413, ...,  0.11161373,
        -0.25409758, -0.03986065],
       [-1.95235299,  1.16765143,  0.13485157, ...,  0.32211166,
         0.04093508, -0.76477956],
       [-1.76966433, -0.21423662, -0.57425174, ...,  1.70790009,
         0.63273017, -0.37517276]])

In [3]:
centroid, label = kmeans2(A, 2, minit='points')
for i in range(10):
    print(label[i])

0
0
1
1
1
1
1
1
0
1


In [4]:
counts = np.bincount(label)
counts

array([3, 7], dtype=int64)

In [5]:
def partition(A):
    centroid, label = kmeans2(A, 2, minit='points')
    A0,A1 = [],[]
    for i in range(10):
        if label[i] == 0:
            A0.append(A[i,:])
        else:
            A1.append(A[i,:])
    A0 = np.array(A0)
    A1 = np.array(A1)
    return (A0,A1)

In [6]:
x,y = partition(A)
num_row,num_col = x.shape
num_row

9

In [8]:
class Node:
    def __init__(self, value):
        self.left = None
        self.data = value
        self.right = None
        #self.children = []
        
    def PrintTree(self):
        if self.left:
            self.left.PrintTree()
        print(self.data),
        if self.right:
            self.right.PrintTree()
            
root = Node(A)
root.PrintTree()

[[-0.06051571  1.81421476 -1.80176172 ...  0.75251048  0.55390197
   0.32961278]
 [-1.19496629 -1.70335251  0.45493366 ... -1.48885837 -0.46498978
  -0.9305189 ]
 [ 0.06033165 -0.80784667  1.16156487 ... -0.34948278 -1.82583457
  -0.53140815]
 ...
 [ 0.62221719  0.68916232  2.19524539 ...  1.16642529  0.30434166
   0.14012133]
 [ 0.43802122 -1.733677    0.13354434 ... -0.73235339 -0.28527902
   0.18751389]
 [ 1.25336017 -0.52650336 -1.61513716 ...  2.060299    0.60011176
  -0.99184089]]


In [34]:
tau = 3
def build(root):
    numrow,numcol = root.data.shape
    if numrow <= tau:
        return root
    A1,A2 = partition(root.data)
    root1 = Node(A1)
    root2 = Node(A2)
    #L = build(root1)
    #R = build(root2)
    root.left = build(root1)
    root.right = build(root2)
    #Node(s).children = [L,R]
    return root

In [48]:
def ourprint(some_root):
        if some_root.left:
            ourprint(some_root.left)
        print(some_root.data)
        if some_root.right:
            ourprint(some_root.right)
#ourprint(root)


In [7]:
# now we will test the above methods using 10 points each in 2 dimension
B = np.random.normal(0, 1.0, (10,2))
new_root = Node(B)
numrow,numcol = new_root.data.shape
numrow

NameError: name 'Node' is not defined

In [7]:
a,b = partition(new_root.data)
new_root1 = Node(a)
new_root2 = Node(b)
n,m = new_root1.data.shape
#print(new_root1.data)
#print(new_root2.data)
#print(a)
#print(b)
new_root.left = new_root1
new_root.right = new_root2

In [8]:
new_root.left.data == new_root1.data

array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True]])

In [66]:
import random
class Rectangle:
    def __init__ (self, width, height, parent = None):
        self.width = width
        self.height = height
        self.parent = parent
        self.children = []

    @property
    def level (self):
        return 0 if not self.parent else 1 + self.parent.level

    def split (self):
        if self.children: raise Exception ('Already split')
        ratio = random.random () * .5 + .25 #split between 1/4 and 3/4
        if self.width > self.height:
            width = int (ratio * self.width)
            self.children = [Rectangle (width, self.height, self),
                Rectangle (self.width - width, self.height, self) ]
        else:
            height = int (ratio * self.height)
            self.children = [Rectangle (self.width, height, self),
                Rectangle (self.width, self.height - height, self) ]

    def splitUntilLevel (self, maxLevel):
        if maxLevel <= self.level: return
        self.split ()
        for child in self.children: child.splitUntilLevel (maxLevel)

    def __str__ (self):
        s = '{}{} x {}\n'.format (' ' * (2 * self.level), self.width, self.height)
        for child in self.children: s += str (child)
        return s

r = Rectangle (100, 100)
r.splitUntilLevel (2)
print (r)

100 x 100
  100 x 32
    29 x 32
    71 x 32
  100 x 68
    32 x 68
    68 x 68



In [11]:
import random
class Node:
    def __init__ (self, data, parent = None):
        #self.width = width
        #self.height = height
        self.parent = parent
        self.children = []
        self.data = data

    @property
    def level (self):
        return 0 if not self.parent else 1 + self.parent.level

    def split (self):
        if self.children == []: return
        numrow,numcol = self.data.shape
        if numrow <= tau:
            self.children = []
        A1,A2 = partition(self.data)
        self.children = [Node(A1, self),Node(A2, self)]
        
    def splitUntilLevel (self, maxLevel):
        if maxLevel <= self.level: return
        self.split ()
        for child in self.children: child.splitUntilLevel (maxLevel)

    def __str__ (self):
        s = "{} x {}".format(self.data.size[0], self.data.size[1])
        for child in self.children: s += str (child)
        return s

N = Node(B)
N.splitUntilLevel (1)
print (N)

IndexError: Replacement index 2 out of range for positional args tuple